In [1]:
import herbie
import xarray
from herbie import Herbie

from datetime import datetime, date, time, timedelta

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
start_date = date(2022, 1, 7)
end_date = date(2022, 1, 14)
times_of_day = [time(00, 0, 00), time(6, 0, 00), time(12, 0, 00), time(18, 0, 00)]

lat_range = slice(60, 30)
lon_range = slice(320, 350)

In [3]:
text = """248     200 mb     UGRD     analysis     U-Component of Wind [m/s]
249     200 mb     VGRD     analysis     V-Component of Wind [m/s]
338     500 mb     UGRD     analysis     U-Component of Wind [m/s]
339     500 mb     VGRD     analysis     V-Component of Wind [m/s]
428     800 mb     UGRD     analysis     U-Component of Wind [m/s]
429     800 mb     VGRD     analysis     V-Component of Wind [m/s]
443     850 mb     UGRD     analysis     U-Component of Wind [m/s]
444     850 mb     VGRD     analysis     V-Component of Wind [m/s]
467     925 mb     HGT     analysis     Geopotential Height [gpm]
468     925 mb     TMP     analysis     Temperature [K]
469     925 mb     RH     analysis     Relative Humidity [%]
473     925 mb     UGRD     analysis     U-Component of Wind [m/s]
474     925 mb     VGRD     analysis     V-Component of Wind [m/s]
512     1000 mb     TMP     analysis     Temperature [K]
513     1000 mb     RH     analysis     Relative Humidity [%]
527     1000 mb     HGT     analysis     Geopotential Height [gpm]
517     1000 mb     UGRD     analysis     U-Component of Wind [m/s]
518     1000 mb     VGRD     analysis     V-Component of Wind [m/s]
528     surface     PRES     analysis     Pressure [Pa]
529     surface     HGT     analysis     Geopotential Height [gpm]
533     surface     LFTX     analysis     Surface Lifted Index [K]
534     surface     CAPE     analysis     Convective Available Potential Energy [J/kg]
535     surface     CIN     analysis     Convective Inhibition [J/kg]"""

In [4]:
result = []
for line in text.split('\n'):
    parts = line.split()
    if parts[1] == 'surface':
        # different to 'xxxx mb' form
        level = parts[1]
        variable = parts[2]
    else:
        # 'xxxx mb' form
        level = parts[1]+' '+parts[2]
        variable = parts[3]
    if f':{variable}:{level}' not in result:
        # append every variable to a list
        result.append(f':{variable}:{level}')
ds = []
for item in result:
    # according to different variable, get database
    ds.append(H.xr(item, remove_grib=False))

NameError: name 'H' is not defined

In [4]:
ds = xarray.Dataset()
for n in range(int((end_date - start_date).days)):
   date = start_date + timedelta(n)
   for time in times_of_day:
      combined_datetime = datetime(date.year, date.month, date.day, time.hour, time.minute, time.second)
      #combined_datetime = datetime.combine(date, time)
      H = Herbie(combined_datetime, model="gfs", product="pgrb2.0p25")
      result = []
      for line in text.split('\n'):
         parts = line.split()
         if parts[1] == 'surface':
            # different to 'xxxx mb' form
            level = parts[1]
            variable = parts[2]
         else:
            # 'xxxx mb' form
            level = parts[1]+' '+parts[2]
            variable = parts[3]
         if f':{variable}:{level}' not in result:
            # append every variable to a list
            result.append(f':{variable}:{level}')
      for item in result:
         print(item)
      # according to different variable, get database
         ds_temp = H.xarray(item, remove_grib=False)
         ds_temp = ds_temp.expand_dims('time')
         #ds_temp = ds_temp.set_coords('time')
         ds_temp = ds_temp.drop_vars('gribfile_projection')
         ds_temp = ds_temp.drop_vars('valid_time')

         if 'surface' in ds_temp.coords:
            suffix = '_surface'
            ds_temp = ds_temp.drop_vars('surface')
            ds_temp = ds_temp.rename({var_name: var_name + suffix for var_name in ds_temp.data_vars})
         elif 'isobaricInhPa' in ds_temp.coords:
            suffix = '_' + str(ds_temp['isobaricInhPa'].values)
            ds_temp = ds_temp.drop_vars('isobaricInhPa')
            ds_temp = ds_temp.rename({var_name: var_name + suffix for var_name in ds_temp.data_vars})
         ds_latlon = ds_temp.sel(latitude = lat_range, longitude = lon_range)
         ds = xarray.merge([ds, ds_latlon])

✅ Found ┊ model=gfs ┊ product=pgrb2.0p25 ┊ 2022-Jan-01 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
:UGRD:200 mb
👨🏻‍🏭 Created directory: [/homes/ochapron/data/gfs/20220101]
:VGRD:200 mb
:UGRD:500 mb
:VGRD:500 mb
:UGRD:800 mb
:VGRD:800 mb
:UGRD:850 mb
:VGRD:850 mb
:HGT:925 mb
:TMP:925 mb
:RH:925 mb
:UGRD:925 mb
:VGRD:925 mb
:TMP:1000 mb
:RH:1000 mb
:HGT:1000 mb
:UGRD:1000 mb
:VGRD:1000 mb
:PRES:surface
:HGT:surface
:LFTX:surface
:CAPE:surface
:CIN:surface
✅ Found ┊ model=gfs ┊ product=pgrb2.0p25 ┊ 2022-Jan-01 06:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
:UGRD:200 mb
:VGRD:200 mb
:UGRD:500 mb
:VGRD:500 mb
:UGRD:800 mb
:VGRD:800 mb
:UGRD:850 mb
:VGRD:850 mb
:HGT:925 mb
:TMP:925 mb
:RH:925 mb
:UGRD:925 mb
:VGRD:925 mb
:TMP:1000 mb
:RH:1000 mb
:HGT:1000 mb
:UGRD:1000 mb
:VGRD:1000 mb
:PRES:surface
:HGT:surface
:LFTX:surface
:CAPE:surface
:CIN:surface
✅ Found ┊ model=gfs ┊ product=pgrb2.0p25 ┊ 2022-Jan-01 12:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
:UGRD:200 mb
:VGRD:200 mb
:UGRD:500 mb
:VGRD:500 mb
:UGRD

In [5]:
ds

<xarray.Dataset>
Dimensions:       (time: 24, latitude: 121, longitude: 121)
Coordinates:
  * time          (time) datetime64[ns] 2022-01-01 ... 2022-01-06T18:00:00
  * latitude      (latitude) float64 60.0 59.75 59.5 59.25 ... 30.5 30.25 30.0
  * longitude     (longitude) float64 320.0 320.2 320.5 ... 349.5 349.8 350.0
    step          timedelta64[ns] 00:00:00
Data variables: (12/23)
    u_200.0       (time, latitude, longitude) float32 -2.683 -2.683 ... 0.4291
    v_200.0       (time, latitude, longitude) float32 -2.704 -3.004 ... -6.574
    u_500.0       (time, latitude, longitude) float32 -4.29 -4.19 ... -1.542
    v_500.0       (time, latitude, longitude) float32 -11.62 -10.62 ... -12.65
    u_800.0       (time, latitude, longitude) float32 -13.02 -12.23 ... -5.158
    v_800.0       (time, latitude, longitude) float32 -18.3 -19.35 ... -9.297
    ...            ...
    v_1000.0      (time, latitude, longitude) float32 -20.35 -20.01 ... -19.89
    sp_surface    (time, latitude, longitude) float32 1.001e+05 ... 1.015e+05
    orog_surface  (time, latitude, longitude) float32 0.05937 ... 0.05937
    lftx_surface  (time, latitude, longitude) float32 11.23 10.83 ... 4.816
    cape_surface  (time, latitude, longitude) float32 43.0 37.0 32.0 ... 0.0 0.0
    cin_surface   (time, latitude, longitude) float32 0.3957 0.3957 ... 0.2982

In [6]:
ds.to_netcdf('/homes/ochapron/data/20210101.nc')

In [7]:
d = xarray.open_dataset('/homes/ochapron/data/20210101.nc')

In [8]:
d

<xarray.Dataset>
Dimensions:       (time: 24, latitude: 121, longitude: 121)
Coordinates:
  * time          (time) datetime64[ns] 2022-01-01 ... 2022-01-06T18:00:00
  * latitude      (latitude) float64 60.0 59.75 59.5 59.25 ... 30.5 30.25 30.0
  * longitude     (longitude) float64 320.0 320.2 320.5 ... 349.5 349.8 350.0
    step          timedelta64[ns] ...
Data variables: (12/23)
    u_200.0       (time, latitude, longitude) float32 ...
    v_200.0       (time, latitude, longitude) float32 ...
    u_500.0       (time, latitude, longitude) float32 ...
    v_500.0       (time, latitude, longitude) float32 ...
    u_800.0       (time, latitude, longitude) float32 ...
    v_800.0       (time, latitude, longitude) float32 ...
    ...            ...
    v_1000.0      (time, latitude, longitude) float32 ...
    sp_surface    (time, latitude, longitude) float32 ...
    orog_surface  (time, latitude, longitude) float32 ...
    lftx_surface  (time, latitude, longitude) float32 ...
    cape_surface  (time, latitude, longitude) float32 ...
    cin_surface   (time, latitude, longitude) float32 ...

In [7]:
da = ds[19]
db = ds[0]
db
for da in ds:
    da = da.drop_vars('gribfile_projection')
    if 'surface' in da.coords:
        suffix = '_surface'
        da = da.drop_vars('surface')
        da = da.rename({var_name: var_name + suffix for var_name in da.data_vars})
    elif 'isobaricInhPa' in da.coords:
        suffix = '_' + str(da['isobaricInhPa'].values)
        da = da.drop_vars('isobaricInhPa')
        da = da.rename({var_name: var_name + suffix for var_name in da.data_vars})



In [8]:
ds

[<xarray.Dataset>
 Dimensions:              (latitude: 721, longitude: 1440)
 Coordinates:
     time                 datetime64[ns] 2021-02-26
     step                 timedelta64[ns] 00:00:00
     isobaricInhPa        float64 200.0
   * latitude             (latitude) float64 90.0 89.75 89.5 ... -89.75 -90.0
   * longitude            (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
     valid_time           datetime64[ns] ...
 Data variables:
     u                    (latitude, longitude) float32 ...
     gribfile_projection  object None
 Attributes:
     GRIB_edition:            2
     GRIB_centre:             kwbc
     GRIB_centreDescription:  US National Weather Service - NCEP
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution:             US National Weather Service - NCEP
     model:                   gfs
     product:                 pgrb2.0p25
     description:             Global Forecast System
     remote_grib:             https://no

In [23]:
merged_ds = xr.merge([ds[0], ds[1]])
merged_ds

<xarray.Dataset>
Dimensions:              (latitude: 721, longitude: 1440)
Coordinates:
    time                 datetime64[ns] 2021-07-11
    step                 timedelta64[ns] 00:00:00
    isobaricInhPa        float64 200.0
  * latitude             (latitude) float64 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time           datetime64[ns] 2021-07-11
Data variables:
    u                    (latitude, longitude) float32 ...
    gribfile_projection  object None
    v                    (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gfs
    product:                 pgrb2.0p25
    description:             Global Forecast System
    remote_grib:             /homes/ochapron/data/gfs/20210711/gfs.t00z.pgrb2...
    local_grib:              /homes/ochapron/data/gfs/20210711/subset_b4ef07b...
    searchString:            :UGRD:200 mb

In [1]:
from herbie import Herbie
H = Herbie("2021-07-11", model="gfs", product="pgrb2.0p25")

✅ Found ┊ model=gfs ┊ product=pgrb2.0p25 ┊ 2021-Jul-11 00:00 UTC F00 ┊ GRIB2 @ local ┊ IDX @ aws


In [38]:
H.inventory()

,grib_message,start_byte,end_byte,range,reference_time,valid_time,variable,level,forecast_time,search_this
0,1,0,878943,0-878943,2021-07-11,2021-07-11,PRMSL,mean sea level,anl,:PRMSL:mean sea level:anl
1,2,878943,1003357,878943-1003357,2021-07-11,2021-07-11,CLWMR,1 hybrid level,anl,:CLWMR:1 hybrid level:anl
2,3,1003357,1168803,1003357-1168803,2021-07-11,2021-07-11,ICMR,1 hybrid level,anl,:ICMR:1 hybrid level:anl
3,4,1168803,1431696,1168803-1431696,2021-07-11,2021-07-11,RWMR,1 hybrid level,anl,:RWMR:1 hybrid level:anl
4,5,1431696,1510779,1431696-1510779,2021-07-11,2021-07-11,SNMR,1 hybrid level,anl,:SNMR:1 hybrid level:anl
...,...,...,...,...,...,...,...,...,...,...
691,692,502099511,502776380,502099511-502776380,2021-07-11,2021-07-11,VGRD,PV=-2e-06 (Km^2/kg/s) surface,anl,:VGRD:PV=-2e-06 (Km^2/kg/s) surface:anl
692,693,502776380,503444054,502776380-503444054,2021-07-11,2021-07-11,TMP,PV=-2e-06 (Km^2/kg/s) surface,anl,:TMP:PV=-2e-06 (Km^2/kg/s) surface:anl
693,694,503444054,504654423,503444054-504654423,2021-07-11,2021-07-11,HGT,PV=-2e-06 (Km^2/kg/s) surface,anl,:HGT:PV=-2e-06 (Km^2/kg/s) surface:anl
694,695,504654423,505823463,504654423-505823463,2021-07-11,2021-07-11,PRES,PV=-2e-06 (Km^2/kg/s) surface,anl,:PRES:PV=-2e-06 (Km^2/kg/s) surface:anl


In [39]:
H.SOURCES

{'aws': 'https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20210711/00/atmos/gfs.t00z.pgrb2.0p25.f000',
 'aws-old': 'https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.20210711/00/gfs.t00z.pgrb2.0p25.f000',
 'ftpprd': 'https://ftpprd.ncep.noaa.gov/data/nccf/com/gfs/prod/gfs.20210711/00/atmos/gfs.t00z.pgrb2.0p25.f000',
 'nomads': 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20210711/00/atmos/gfs.t00z.pgrb2.0p25.f000',
 'google': 'https://storage.googleapis.com/global-forecast-system/gfs.20210711/00/atmos/gfs.t00z.pgrb2.0p25.f000',
 'azure': 'https://noaagfs.blob.core.windows.net/gfs/gfs.20210711/00/atmos/gfs.t00z.pgrb2.0p25.f000'}

In [40]:
H.PRODUCTS

{'pgrb2.0p25': 'common fields, 0.25 degree resolution',
 'pgrb2.0p50': 'common fields, 0.50 degree resolution',
 'pgrb2.1p00': 'common fields, 1.00 degree resolution',
 'pgrb2b.0p25': 'uncommon fields, 0.25 degree resolution',
 'pgrb2b.0p50': 'uncommon fields, 0.50 degree resolution',
 'pgrb2b.1p00': 'uncommon fields, 1.00 degree resolution',
 'pgrb2full.0p50': 'combined grids of 0.50 resolution'}

In [41]:
H.download()

PosixPath('/homes/ochapron/data/gfs/20210711/gfs.t00z.pgrb2.0p25.f000')

In [42]:
ds = H.xarray(":UGRD")

Note: Returning a list of [12] xarray.Datasets because cfgrib opened with multiple hypercubes.


In [35]:
ds[1]

<xarray.Dataset>
Dimensions:              (latitude: 721, longitude: 1440)
Coordinates:
    time                 datetime64[ns] 2021-07-11
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 10.0
  * latitude             (latitude) float64 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 0.0 0.25 0.5 ... 359.2 359.5 359.8
    valid_time           datetime64[ns] 2021-07-11
Data variables:
    u10                  (latitude, longitude) float32 4.628 4.618 ... -6.002
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gfs
    product:                 pgrb2.0p25
    description:             Global Forecast System
    remote_grib:             /homes/ochapron/data/gfs/20210711/gfs.t00z.pgrb2...
    local_grib:              /homes/ochapron/data/gfs/20210711/subset_b4ef146...
    searchString:            :UGRD

In [36]:
ds[1].sel(latitude=lat_range, longitude=lon_range)

<xarray.Dataset>
Dimensions:              (latitude: 121, longitude: 41)
Coordinates:
    time                 datetime64[ns] 2021-07-11
    step                 timedelta64[ns] 00:00:00
    heightAboveGround    float64 10.0
  * latitude             (latitude) float64 60.0 59.75 59.5 ... 30.5 30.25 30.0
  * longitude            (longitude) float64 0.0 0.25 0.5 0.75 ... 9.5 9.75 10.0
    valid_time           datetime64[ns] 2021-07-11
Data variables:
    u10                  (latitude, longitude) float32 5.148 4.898 ... -0.05162
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gfs
    product:                 pgrb2.0p25
    description:             Global Forecast System
    remote_grib:             /homes/ochapron/data/gfs/20210711/gfs.t00z.pgrb2...
    local_grib:              /homes/ochapron/data/gfs/20210711/subset_b4ef146...
    searchString:            :UGRD

In [20]:
type(ds)

xarray.core.dataset.Dataset

In [18]:
ax = EasyMap(crs=ds.herbie.crs, figsize=[10, 8]).ax
p = ax.pcolormesh(
    ds.longitude, ds.latitude, ds.t2m, transform=pc, **cm_tmp(units="K").cmap_kwargs
)
plt.colorbar(
    p, ax=ax, orientation="horizontal", pad=0.05, **cm_tmp(units="K").cbar_kwargs
)

ax.set_title(ds.t2m.GRIB_name, loc="right")
ax.set_title(f"{ds.model.upper()}: {H.product_description}", loc="left")

NameError: name 'EasyMap' is not defined

In [4]:
import cfgrib
ds = herbie.load_dataset(H)
ds = H.xarray("TMP:2 m", engine = 'cfgrib')

AttributeError: module 'herbie' has no attribute 'load_dataset'

In [1]:
import xarray as xr
import datetime as dt
from itertools import chain
import numpy as np

In [2]:
range1 = lambda start, end, step=1: np.arange(start, end + step, step)
date = (dt.date.today() - dt.timedelta(days=1)).strftime("%Y%m%d")
gfs = f"https://nomads.ncep.noaa.gov/dods/gfs_0p25_1hr"

lat = (-2, 2)
# Since lon is in the range 0, 360 and not -180, 180 it is common wanting to download a non-consecutive interval
# For instance 0, 0.25 and 358,359.75. This would correspond to -2, 0.25 in -180, 180 range
# Then we cannot use a slice object, we need to explicitily list all values
lon = list(chain(range1(0, 2, 0.25), range1(358, 359.75, 0.25)))
time = (0, 1)
var = ["tmp2m"]
runs = [0]

print(date)
print(lon)

for run in runs:
    url = f"{gfs}/gfs{date}/gfs_0p25_1hr_{run:02d}z"
    print(url)
    with xr.open_dataset(url) as ds:
        (
            ds[var]
            .isel(time=slice(*time))
            .sel(lat=slice(*lat), lon=lon)
            .to_netcdf(f"{date}_{run:02d}.nc")
        )


20231116
[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 358.0, 358.25, 358.5, 358.75, 359.0, 359.25, 359.5, 359.75]
https://nomads.ncep.noaa.gov/dods/gfs_0p25_1hr/gfs20231116/gfs_0p25_1hr_00z


/DATASET/envs/oscar/miniconda/envs/4dvarnet-starter/lib/python3.9/site-packages/xarray/coding/times.py:154: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)
/DATASET/envs/oscar/miniconda/envs/4dvarnet-starter/lib/python3.9/site-packages/xarray/coding/times.py:154: SerializationWarning: Ambiguous reference date string: 1-1-1 00:00:0.0. The first value is assumed to be the year hence will be padded with zeros to remove the ambiguity (the padded reference date string is: 0001-1-1 00:00:0.0). To remove this message, remove the ambiguity by padding your reference date strings with zeros.
  warnings.warn(warning_msg, SerializationWarning)


In [3]:
gfs = xr.open_dataset('/homes/ochapron/Documents/OH-08-BR2/code/20231116_00.nc')

In [4]:
gfs

<xarray.Dataset>
Dimensions:  (time: 1, lat: 17, lon: 17)
Coordinates:
  * time     (time) datetime64[ns] 2023-11-16
  * lat      (lat) float64 -2.0 -1.75 -1.5 -1.25 -1.0 ... 1.0 1.25 1.5 1.75 2.0
  * lon      (lon) float64 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Data variables:
    tmp2m    (time, lat, lon) float32 ...
Attributes:
    title:        GFS 0.25 deg starting from 00Z16nov2023, downloaded Nov 16 ...
    Conventions:  COARDS\nGrADS
    dataType:     Grid
    history:      Fri Nov 17 17:47:37 UTC 2023 : imported by GrADS Data Serve...

In [5]:
#!/usr/bin/env python
import datetime as dt
import logging

import typer
import xarray as xr

from utils import set_logging

GFS_HIST_BASE = "https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-004-files"


def get_gfs_hist(
    date: dt.date,
    varlist: list,
    run: int = 0,
    time: int = 0,
):

    date_str = date.strftime("%Y%m%d")
    month_str = date.strftime("%Y%m")
    url = f"{GFS_HIST_BASE}/{month_str}/{date_str}/gfs_3_{date_str}_{run:02d}00_{time:03d}.grb2"

    logging.info(url)

    with xr.open_dataset(url) as ds:
        ds[varlist].to_netcdf(f"{date_str}_{run:02d}_{time:03d}.nc")


def main(date: dt.datetime = None, time: int = 0, run: int = 0, log: str = "info"):

    set_logging(log)

    date = date.date() if date is not None else dt.date.today() - dt.timedelta(days=30)

    variables = [
        "u-component_of_wind_height_above_ground",
        "v-component_of_wind_height_above_ground",
    ]
    try:
        get_gfs_hist(date, variables, time=time, run=run)
    except Exception as err:
        logging.exception(err)


if __name__ == "__main__":
    typer.run(main)

ModuleNotFoundError: No module named 'typer'